**Table of contents**<a id='toc0_'></a>    
- 1. [IMPORTS](#toc1_)    
- 2. [CREATE INITIAL TE DATAFRAME](#toc2_)    
  - 2.1. [FX: `build_TE_df_from_gff3`](#toc2_1_)    
- 3. [IMPORT CHROMOSOME FILES AND GENERATE CHROMOSOME DATAFRAMES](#toc3_)    
  - 3.1. [FXS: Get chromosome information](#toc3_1_)    
    - 3.1.1. [`get_chr_file`](#toc3_1_1_)    
    - 3.1.2. [`fix_chr_names`](#toc3_1_2_)    
    - 3.1.3. [`get_relative_positions`](#toc3_1_3_)    
    - 3.1.4. [`make_chr_df_from_fasta`](#toc3_1_4_)    
  - 3.2. [FX: `assign_sequence`](#toc3_2_)    
- 4. [IMPORT BEDTOOLS INTERSECT FILE](#toc4_)    
  - 4.1. [FX: `create_bedtools_dict(intersect_file)`](#toc4_1_)    
- 5. [PARSE SNP .VCF](#toc5_)    
  - 5.1. [FX: `import_sort_vcf`](#toc5_1_)    
  - 5.2. [FX: `get_alternate_sequences(N2_dataframe, teswsnps_dict)`](#toc5_2_)    
- 6. [LOOK FOR ALTERNATE SEQUENCES IN CB reasonaTE RESULTS](#toc6_)    
  - 6.1. [FX: `get_CB_te_seq(CB_dataframe)`](#toc6_1_)    
  - 6.2. [FX: `get_matching_unique_tes(alt_seqs_dict, CB_dataframe)`](#toc6_2_)    
  - 6.3. [FX: `split_dup_copies`](#toc6_3_)    
- 7. [FIND TEs](#toc7_)    
  - 7.1. [BEDTOOLS INTERSECT](#toc7_1_)    
  - 7.2. [GENERATE ALT SEQS AND FIND TEs](#toc7_2_)    
- 8. [CREATE TSV FILE](#toc8_)    
  - 8.1. [FX: `create_tsv_file`](#toc8_1_)    
  - 8.2. [FX: `create_tsv_file_specfams`](#toc8_2_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[IMPORTS](#toc0_)

In [27]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
from matplotlib import cm
import seaborn as sns
sns.set(style="whitegrid")
from Bio import SeqIO
from Bio.Seq import Seq
import csv
# from joypy import joyplot
# from matplotlib_venn import venn2, venn2_circles, venn3, venn3_circles
# from lolviz import *


/var/folders/60/m3wm021x08vbp7kk3s0t94c40000gn/T/ipykernel_91768/3365580798.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# 2. <a id='toc2_'></a>[CREATE INITIAL TE DATAFRAME](#toc0_)
## 2.1. <a id='toc2_1_'></a>[FX: `build_TE_df_from_gff3`](#toc0_)
- import .gff3, calculate TE sizes and add in seq records, create dataframe, sort dataframe first by chromosome name and then start position
- make sure to check the names of your chromosomes in your .gff3 file match the names in the `gff_chrom_id` list in the function so that they are replaced with the TEUlt names correctly

In [28]:
def build_TE_df_from_gff3(gff3_file: str):
    """ 
    gff3_file: str, path to .gff3 output file from TEUlt
    """
    
    gff_chrom_id = ['seq1', 'seq2', 'seq3', 'seq4', 'seq5', 'seq6']
    chromosomes = ['Chr I', 'Chr II', 'Chr III', 'Chr IV', 'Chr V', 'X Chr']
    
    gff_list = []
    
    with open(gff3_file, newline = '') as lines:                #open gff file, take as tab delimited, create huge list that is the gff file
        line_reader = csv.reader(lines, delimiter='\t')
        for line in line_reader:
                gff_list.append(line)

    gff_data = []
    
    for i in range(len(gff_list)):
        
#         print(i)
        temp = [ gff_list[i][0], int(gff_list[i][3]), int(gff_list[i][4]), gff_list[i][5], gff_list[i][6] ] #chromosome, start, end, score, strand...
        
        id_start = gff_list[i][8].find('=') +1
        id_stop = gff_list[i][8].find(';') 
        te_id = int(gff_list[i][8][id_start:id_stop]) # get TE id
        temp.append(te_id)
        
        size = int(gff_list[i][4]) - int(gff_list[i][3]) #get TE size (stop_pos - start_pos) 
        temp.append(size)
        
        class_start = gff_list[i][8].find('(') +1
        class_end = gff_list[i][8].find(')')
        class_info = gff_list[i][8][class_start:class_end].split(',') #find transposon class info/descriptions 
        
        if len(class_info) == 3: ##if not a helitron...
            
            family = class_info[0]
            sub_class = class_info[1]
            main_class = class_info[2]


            temp.append(family)
            temp.append(sub_class)
            temp.append(main_class) ##add these three class/ID fields to the temp line
            
        elif len(class_info) == 2: #if a helitron...
            
            family = class_info[0]
            sub_class = '...'
            main_class = class_info[1]


            temp.append(family)
            temp.append(sub_class)
            temp.append(main_class) ##add these three class/ID fields to the temp line
            
        
        gff_data.append(temp) ##append each "line" to data list of lists
      
    gff_df = pd.DataFrame(gff_data, columns=["SeqChr", "Start", "Stop", "Score:Probability", "Strand", 'ID', 'Size', 'Family', 'Subclass', 'Class']) #turn list of lists into dataframe
    
      

    gff_df['Chromosome'] = gff_df['SeqChr']

    
    for i in range(6):
        
        gff_df['Chromosome'] = gff_df['Chromosome'].replace(gff_chrom_id[i], chromosomes[i]) ##replace TransposonUltimate chromosome names with something better
  
        
    gff_df_sorted = gff_df.sort_values(by=['Chromosome', 'Start'], axis=0, ascending=True, ignore_index=True) ##sort by chromosome name first, then starting position within each chromosome, reindex rows...
    
    return gff_df_sorted

# 3. <a id='toc3_'></a>[IMPORT CHROMOSOME FILES AND GENERATE CHROMOSOME DATAFRAMES](#toc0_)
## 3.1. <a id='toc3_1_'></a>[FXS: Get chromosome information](#toc0_)
### 3.1.1. <a id='toc3_1_1_'></a>[`get_chr_file`](#toc0_)
- import chr file and make df
- this function is specific to the chr_lengths.txt file included, and could also be easily altered to accept more or less genomes within the chr_lengths.txt file (ex. 4 genomes instead of 2)
### 3.1.2. <a id='toc3_1_2_'></a>[`fix_chr_names`](#toc0_)
- changes chromosome names to the names used by TEUlt
- change the `old_chr_names` list within the function to the names of your chromosomes

### 3.1.3. <a id='toc3_1_3_'></a>[`get_relative_positions`](#toc0_)
- calculates the relative start positions of transposons as a fraction of the length of the genome using the chromosome lengths from chr_df
- allows for comparison of position of TEs between chromosomes with different lengths

### 3.1.4. <a id='toc3_1_4_'></a>[`make_chr_df_from_fasta`](#toc0_)
- makes the needed chromosome dataframe from a genome's fasta file (separates out chromosome sequences and gets the chromosome, start and stop positions)

In [29]:
def get_chr_file(chr_lengths_file: str):
    chr_lines_list = []
    with open(chr_lengths_file, newline = '') as chr_lines:
        chr_line_reader = csv.reader(chr_lines, delimiter='\t')
        for line in chr_line_reader:
            chr_lines_list.append(line)
            
    N2_chr_list = []
    CB_chr_list = []
    for i in range(len(chr_lines_list)):
        
        if chr_lines_list[i][0] == 'Chr':
            continue
            
        elif chr_lines_list[i][3] == 'Libuda N2':
            chr_lines_list[i][2] = int(chr_lines_list[i][2])
            N2_chr_list.append(chr_lines_list[i])
            
        elif chr_lines_list[i][3] == 'Libuda CB':
            chr_lines_list[i][2] = int(chr_lines_list[i][2])
            CB_chr_list.append(chr_lines_list[i])
            
    N2_chr_df = pd.DataFrame(N2_chr_list, columns = ['chr', 'start', 'end', 'strain'])
    
    CB_chr_df = pd.DataFrame(CB_chr_list, columns = ['chr', 'start', 'end', 'strain'])
            
    return N2_chr_df, CB_chr_df

def fix_chr_names(dataframe):
    
    old_chr_names = ['I', 'II', 'III', 'IV', 'V', 'X']

    for index, row in dataframe.iterrows():
        if dataframe.at[index, 'chr'] == old_chr_names[0]:
            dataframe.at[index, 'chr'] = 'Chr I'
        elif dataframe.at[index, 'chr'] == old_chr_names[1]:
            dataframe.at[index, 'chr'] = 'Chr II'
        elif dataframe.at[index, 'chr'] == old_chr_names[2]:
            dataframe.at[index, 'chr'] = 'Chr III'
        elif dataframe.at[index, 'chr'] == old_chr_names[3]:
            dataframe.at[index, 'chr'] = 'Chr IV'
        elif dataframe.at[index, 'chr'] == old_chr_names[4]:
            dataframe.at[index, 'chr'] = 'Chr V'
        elif dataframe.at[index, 'chr'] == old_chr_names[5]:
            dataframe.at[index, 'chr'] = 'X Chr'
            
def get_relative_positions(all_df, chr_df):
    for index, row in all_df.iterrows():
        for cindex, crow in chr_df.iterrows():
            if (all_df.at[index, 'Chromosome']) == (chr_df.at[cindex, 'chr']): # if the same chr
                all_df.at[index, 'Relative Start Position'] = int(all_df.at[index, 'Start']) / int(chr_df.at[cindex, 'end']) #

def make_chr_df_from_fasta(fasta_file: str, species: str):
    info = []
    counter = 0
    for record in SeqIO.parse(fasta_file, "fasta"):
        
        chr_name = chromosomes_roman[kim_counter]
        start = 1
        end = len(record)
        species = species
        
        info.append([chr_name, start, end, species])
        counter = counter + 1
    #     print("%s %i" % (record.id, len(record)))

    chr_df = pd.DataFrame(info, columns = ['chr', 'start', 'end', 'strain'])
    return chr_df


## 3.2. <a id='toc3_2_'></a>[FX: `assign_sequence`](#toc0_)
- add seq chr names and reindex according to TE ID

In [1]:
# from detect analysis notebook
# assign seqio records to each TE so that you can get the sequence of the TE later


def assign_sequence(fasta_file: str, dataframe, column_name: str):
    """ 
    fasta_file: str, filepath of genome fasta file
    dataframe: TE dataframe
    column_name: str, name of column that seqio records will be put into
    """
    
    gen = ['']*len(dataframe.Start)
    dataframe[column_name] = gen


    for record in SeqIO.parse(fasta_file, "fasta"):

        print(record.id) #sanity chek

        for index, row in dataframe.iterrows():

            if dataframe.at[index, 'SeqChr'] == str(record.id):
                
                dataframe.at[index, column_name] = record
    
    dataframe2 = dataframe.set_index('ID')
                
    return dataframe2



# 4. <a id='toc4_'></a>[IMPORT BEDTOOLS INTERSECT FILE](#toc0_)
## 4.1. <a id='toc4_1_'></a>[FX: `create_bedtools_dict(intersect_file)`](#toc0_)
- import the outfile from bedtools intersect and create a dictionary and dataframe with the necessary information from that file
- change the `old_chrom_names` list so that when imported, the chromosome names will be replaced correctly

In [31]:
def create_bedtools_dict(intersect_file: str):
    """
    intersect_file: str, path to bedtools intersect outfile 
    """

    with open(intersect_file) as f:
        lines = f.readlines()

        lines_list = []
        for line in lines: 
            line1 = line.split('\t')
            lines_list.append(line1)
    
    df_list = []
    
    old_chr_names = ['N2_chrI', 'N2_chrII', 'N2_chrIII', 'N2_chrIV', 'N2_chrV', 'N2_chrX']

    for i in range(len(lines_list)):
        chrom = lines_list[i][0]
        if chrom == old_chr_names[0]:
            chrom = 'Chr I'
        elif chrom == old_chr_names[1]:
            chrom = 'Chr II'
        elif chrom == old_chr_names[2]:
            chrom = 'Chr III'
        elif chrom == old_chr_names[3]:
            chrom = 'Chr IV'
        elif chrom == old_chr_names[4]:
            chrom = 'Chr V'
        elif chrom == old_chr_names[5]:
            chrom = 'X Chr'
    


        id_start = lines_list[i][8].find('=') +1
        id_stop = lines_list[i][8].find(';') 
        te_id = int(lines_list[i][8][id_start:id_stop]) # get TE id

        position = lines_list[i][10]

        reference = lines_list[i][12]
        alternate = lines_list[i][13]

        df_list.append([te_id, [chrom, position, reference, alternate]])

    tes_with_snps_df = pd.DataFrame(df_list, columns = ['id', 'snps'])   
    
    tes_with_snps_df = tes_with_snps_df.groupby(['id']).agg(tuple).applymap(list).reset_index()
    
    tes_with_snps_dict = dict(zip(tes_with_snps_df.id, tes_with_snps_df.snps))
    
    return [tes_with_snps_dict, tes_with_snps_df]


# 5. <a id='toc5_'></a>[PARSE SNP .VCF](#toc0_)
## 5.1. <a id='toc5_1_'></a>[FX: `import_sort_vcf`](#toc0_)
+ import the vcf file and turn each line into a list within a larger list
+ sort vcf, change chromosome names to the same ones as gff3 and then make it into a dataframe
+ if the naming scheme in your vcf file is not 'Chr I', 'Chr II', 'Chr III' etc, there is a piece of this function that will automatically change the chromosome names to align with the naming scheme used by TE Ult, you will just have to alter the `old_chr_names` list to match the names of your chromosomes in your vcf

In [32]:
def import_sort_vcf(filename: str): #import the vcf file and turn each line into a list in a larger list
    """ 
    filename: str, filepath of vcf file
    """
    with open(filename) as f:
        lines = f.readlines()
        
        lines_list = []
        for line in lines: 
            line_list = line.split('\t')
            lines_list.append(line_list)
            
            
    header_lines_list = []
    info_lines_list = []
    
    for indiv_line in lines_list:
#         indiv_line = lines_list[i]
        if indiv_line[0][0] != '#':
            info_lines_list.append(indiv_line[0:5])
        else:
            header_lines_list.append(indiv_line)
    
            
    snp_df = pd.DataFrame(info_lines_list, columns = ['Chromosome', 'Position', 'blank', 'N2 Reference', 'CB Alternate'])
    snp_df = snp_df.reindex(columns = ['Chromosome', 'Position', 'N2 Reference', 'CB Alternate'])
    
    old_chr_names = ['N2_chrI', 'N2_chrII', 'N2_chrIII', 'N2_chrIV', 'N2_chrV', 'N2_chrX']
    chromosomes = ['Chr I', 'Chr II', 'Chr III', 'Chr IV', 'Chr V', 'X Chr']

    for i in range(6):
        
        snp_df['Chromosome'] = snp_df['Chromosome'].replace(old_chr_names[i], chromosomes[i]) ##replace TransposonUltimate chromosome names with something better
    
    return snp_df    
        
            
    
            

## 5.2. <a id='toc5_2_'></a>[FX: `get_alternate_sequences(N2_dataframe, teswsnps_dict)`](#toc0_)
- merge the SNPs (CB relative to N2) and N2 TE annotation data to get the alternate TE sequences that are expected to be seen in the CB TE annotations

In [33]:
def get_alternate_sequences(N2_dataframe, teswsnps_dict):
    
    alt_seqs = {}
    

    for te_id, value in teswsnps_dict.items(): # for each TE that has SNPs (each TE in the dictionary returned from find_tes_with_snps)
        
        sequence_record = N2_dataframe.loc[te_id, 'N2 record']

        chr_seq = str(sequence_record.seq) # get chromosome sequence

        te_start = int(N2_dataframe.loc[te_id, 'Start']) # get TE start and stop coords
        
        te_stop = int(N2_dataframe.loc[te_id, 'Stop'])

        N2_te_seq = (chr_seq[te_start-1 : te_stop-1])

        CB_seq = N2_te_seq # create CB sequence variable

        for i in range(len(value)):

            snp_pos = int(value[i][1]) # get snp position
            alt_pos = (snp_pos - te_start) # get position of snp relative to short te sequence

            # need to always get first snp, can't go by 0th index because there are some rando ones that are sequences and not just one letter
            if len(value[i][3]) == 1:
                snp_alt_allele = value[i][3][0] # get alternate allele if there is only one available
            elif len(value[i][3]) > 1:
                alt_allele_list = value[i][3].split(',') # if a sequence or more than one allele option, split the options at the comma and get the first one
                snp_alt_allele = alt_allele_list[0]

            CB_seq = CB_seq[:alt_pos] + str(snp_alt_allele) + CB_seq[alt_pos+1:] #replace reference allele with alternate allele and leave the rest of the TE sequence intact

        alt_seqs[te_id] = CB_seq

    return alt_seqs


# 6. <a id='toc6_'></a>[LOOK FOR ALTERNATE SEQUENCES IN CB reasonaTE RESULTS](#toc0_)

## 6.1. <a id='toc6_1_'></a>[FX: `get_CB_te_seq(CB_dataframe)`](#toc0_)
- get the CB TE sequences and add the sequences to the CB dataframe
- **only have to run once**

In [34]:
def get_CB_te_seq(CB_dataframe):
    
    df_length = ['']*len(CB_dataframe.Start)
    CB_dataframe['CB TE seq'] = df_length

    for index, row in CB_dataframe.iterrows():

        cb_record = CB_dataframe.at[index, 'CB record']

        start_pos = int(CB_dataframe.at[index, 'Start']) - 1 # get CB TE start and stop positions, and convert to coords that get correct sequence from seq record (because 1 -> 0 base)
        stop_pos = int(CB_dataframe.at[index, 'Stop']) - 1


        cb_chr_seq = str(cb_record.seq) # convert chromosome sequence to string
        cb_te_seq = cb_chr_seq[start_pos:stop_pos] # get TE sequence from chromosome string using start and stop coords

        CB_dataframe.at[index, 'CB TE seq'] = cb_te_seq


    return CB_dataframe


## 6.2. <a id='toc6_2_'></a>[FX: `get_matching_unique_tes(alt_seqs_dict, CB_dataframe)`](#toc0_)
- gets the N2 and CB ids of the matching TEs
- outputs the ids into a .tsv file called matching_tes_withdups.tsv

In [35]:
def get_matching_unique_tes(alt_seqs_dict, CB_dataframe):
    
    with open('matchingTEs_IDs_withdups.tsv', 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        
        matching_list = []

        for n2_te_id, alt_seq in alt_seqs_dict.items(): # for each unique TE
            
#             print(n2_te_id, alt_seq)

            if alt_seq in CB_dataframe['CB TE seq'].values.tolist(): # if the sequence was found by reasonaTE in CB

                cb_te_id = (CB_dataframe[CB_dataframe['CB TE seq'] == alt_seq].index.values) # get the CB TE id from reasonaTE results

                matching_list.append([n2_te_id, cb_te_id])

                print('N2 ID: ', n2_te_id, 'CB ID: ', cb_te_id)
                
                tsv_writer.writerow([n2_te_id, cb_te_id])

    return matching_list



    
   

## 6.3. <a id='toc6_3_'></a>[FX: `split_dup_copies`](#toc0_)
- if after locating unique TE copies that have moved, there are copies that are found in multiple locations, this function will split those TE ids into their own separate lines 

ex. 
 
**original** \
[146, [148, 151]] \
[1251, [1234, 1235]]

**split dups** \
[146, 148] \
[146, 151] \
[1251, 1234] \
[1251, 1235] 

- outputs a file called matching_tes_splitdups.tsv that has the matching TE ids but TEs that have one copy in N2 and more than one in CB are on separate lines


In [36]:
def split_dup_copies(matching_list: list):
    """ 
    matching_list: list, output list from `get_matching_unique_tes`
    """
    matching_list2 = matching_list.copy()

    for i in range(len(matching_list2)):
        
        matching_list2[i][1] = matching_list2[i][1].tolist()
        temp = [int(item) for item in matching_list2[i][1]]
        matching_list2[i][1] = temp
        
        if len(matching_list2[i][1]) > 1:  # creates entry for every cb copy (ex. if there are 2 copies of alternate sequence from 1 n2 id, there will be 2 entries -> one for each cb copy)
            
            copies = matching_list2[i][1]
            print(matching_list2[i])
                    
            matching_list2[i][1] = copies[0]
                        
            matching_list2.append([int(matching_list2[i][0]), int(copies[1])])
                
            
        else:
            matching_list2[i][1] = matching_list2[i][1][0]

#     for j in range(len(matching_list2[i][1])): # converts each cb id value into integer
        
#         matching_list2[i][1][j] = int(matching_list2[i][1][j])


    with open('matchingTEs_IDs_splitdups.tsv', 'wt') as out_file:
        
        tsv_writer = csv.writer(out_file, delimiter='\t')

        tsv_writer.writerows(matching_list2)

    return matching_list2

# 7. <a id='toc7_'></a>[FIND TEs](#toc0_)

## 7.1. <a id='toc7_1_'></a>[BEDTOOLS INTERSECT](#toc0_)
- this cell will run the bedtools intersect but you can also run it in your own terminal or whatever is best for you!
- the function that imports the bedtools intersect outfile is written to read the output from the following bedtools function call: \
  `bedtools intersect -wa -wb -a FinalAnnotations_Transposons.gff3 -b CB.aligned.to.N2.SNPs.final.seqnames.vcf`

In [37]:
# %%bash
# bedtools intersect -wa -wb -a N2_filter.gff3 -b "CB.aligned.to.N2.SNPs.final.seqnames.vcf" > N2TEs_CBSNPs_intersect.txt

## 7.2. <a id='toc7_2_'></a>[GENERATE ALT SEQS AND FIND TEs](#toc0_)

In [38]:
intersect_file = 'N2TEs_CBSNPs_intersect.txt'

tes_with_snps_list = create_bedtools_dict(intersect_file)
tes_with_snps_dict = tes_with_snps_list[0]
tes_with_snps_df = tes_with_snps_list[1]
# print((tes_with_snps_df))

In [39]:
chr_lengths_file = 'chr_lengths.txt'

N2_chr_df = get_chr_file(chr_lengths_file)[0]
CB_chr_df = get_chr_file(chr_lengths_file)[1]

fix_chr_names(N2_chr_df)
fix_chr_names(CB_chr_df)

In [40]:
N2_file = 'N2_filter.gff3'
CB_file = 'CB_filter.gff3'

N2_all_df = build_TE_df_from_gff3(N2_file)
CB_all_df = build_TE_df_from_gff3(CB_file)

get_relative_positions(N2_all_df, N2_chr_df)
get_relative_positions(CB_all_df, CB_chr_df)

libN2_fasta = 'N2_sequence.fasta'
libCB_fasta = 'CB_sequence.fasta'

N2_all_df = assign_sequence(libN2_fasta, N2_all_df, 'N2 record')
CB_all_df = assign_sequence(libCB_fasta, CB_all_df, 'CB record')

seq1
seq2
seq3
seq4
seq5
seq6
seq1
seq2
seq3
seq4
seq5
seq6


In [41]:
vcf_file = 'CB.aligned.to.N2.SNPs.final.vcf'

snp_df = import_sort_vcf(vcf_file)

In [42]:
alt_seqs_dict = get_alternate_sequences(N2_all_df, tes_with_snps_dict)

In [43]:
CB_all_df = get_CB_te_seq(CB_all_df)

In [44]:
matching_list = get_matching_unique_tes(alt_seqs_dict, CB_all_df)
print(len(matching_list))

N2 ID:  4 CB ID:  [3]
N2 ID:  19 CB ID:  [19]
N2 ID:  41 CB ID:  [40]
N2 ID:  69 CB ID:  [67]
N2 ID:  79 CB ID:  [77]
N2 ID:  103 CB ID:  [102]
N2 ID:  113 CB ID:  [112]
N2 ID:  115 CB ID:  [114]
N2 ID:  123 CB ID:  [122]
N2 ID:  146 CB ID:  [148 151]
N2 ID:  151 CB ID:  [156]
N2 ID:  178 CB ID:  [182]
N2 ID:  201 CB ID:  [203]
N2 ID:  243 CB ID:  [240]
N2 ID:  244 CB ID:  [241]
N2 ID:  250 CB ID:  [246]
N2 ID:  286 CB ID:  [283]
N2 ID:  287 CB ID:  [284]
N2 ID:  306 CB ID:  [304]
N2 ID:  307 CB ID:  [305]
N2 ID:  341 CB ID:  [333]
N2 ID:  354 CB ID:  [346]
N2 ID:  362 CB ID:  [354]
N2 ID:  396 CB ID:  [381]
N2 ID:  411 CB ID:  [395]
N2 ID:  415 CB ID:  [398]
N2 ID:  423 CB ID:  [406]
N2 ID:  434 CB ID:  [417]
N2 ID:  436 CB ID:  [419]
N2 ID:  447 CB ID:  [429]
N2 ID:  449 CB ID:  [431]
N2 ID:  464 CB ID:  [444]
N2 ID:  490 CB ID:  [473]
N2 ID:  503 CB ID:  [484]
N2 ID:  509 CB ID:  [489]
N2 ID:  511 CB ID:  [491]
N2 ID:  524 CB ID:  [505]
N2 ID:  526 CB ID:  [507]
N2 ID:  541 CB ID:  

In [45]:
matching_list2 = split_dup_copies(matching_list)

[146, [148, 151]]
[1251, [1234, 1235]]
[1773, [1753, 1752]]
[1795, [1789, 1790]]
[2193, [2163, 2164]]
[2324, [2288, 2844]]
[2457, [675, 2414]]
[4243, [4385, 4390]]
[5404, [5300, 5301]]
[5406, [5302, 5303]]
[28997, [28803, 28804]]
[29831, [29612, 29615]]
[32119, [31823, 31830]]
[32462, [32145, 32150]]
[36059, [35780, 35898]]
[36950, [36595, 36606]]
[36956, [36603, 36614]]
[37125, [36654, 36684]]
[37130, [36658, 36688]]
[37210, [36846, 36859]]
[37260, [36846, 36859]]
[38442, [37865, 37868]]
[38458, [37883, 38851]]


In [46]:
matching_df = pd.DataFrame(matching_list2, columns = ['N2_id', 'CB_id'])
matching_df

,N2_id,CB_id
0,4,3
1,19,19
2,41,40
3,69,67
4,79,77
...,...,...
1527,37130,36688
1528,37210,36859
1529,37260,36859
1530,38442,37868


# 8. <a id='toc8_'></a>[CREATE TSV FILE](#toc0_)
## 8.1. <a id='toc8_1_'></a>[FX: `create_tsv_file`](#toc0_)
- creates tsv file that is ready to be input the R program RIdeogram (documentation [here](https://cran.r-project.org/web/packages/RIdeogram/vignettes/RIdeogram.html)) or input into the code to determine whether the movement of the TEs is real or due to alignment differences
- this function has an option to include the TE IDs in the output tsv file, but in order for the file to be compatible with RIdeogram, the IDs can't be included
- when generating the tsv for determining whether TE movement is real or due to alignment differences, the TE ids should be included

rideogram synteny format \
Species_1 \t Start_1 \t End_1 \t Species_2 \t Start_2 \t End_2 \t fill \
\
N2 chr \t n2 start \t n2 stop \t CB chr \t cb start \t cb stop \t fill

In [52]:
def create_tsv_file(matching_df, sort_by_class: bool, include_ids: bool = False, to_save: bool = False):
    """ 
    matching_df: dataframe created above
    sort_by_class: bool, whether to color the TE lines by class or not
    include_ids: bool, whether to include the TE ids in columns in the output file (RIdeogram will NOT read the file if the ids are included)- when this is set to True, the TE ids will be included and the color column will NOT be included
    to_save: bool, whether or not to save the graph as a pdf (if False, graph will only be output into the notebook)
    """

    df_ids = ['seq1', 'seq2', 'seq3', 'seq4', 'seq5', 'seq6']
    rideogram_ids = [1, 2, 3, 4, 5, 6]

    matching_tsv_list = []
    for index, row in matching_df.iterrows():
        n2_id = int(row['N2_id'])
        cb_id = (row['CB_id'])

        # replace chromosome names with numbers for RIdeogram      
        for i in range(6):
            
            N2_all_df['SeqChr'] = N2_all_df['SeqChr'].replace(df_ids[i], rideogram_ids[i])
            CB_all_df['SeqChr'] = CB_all_df['SeqChr'].replace(df_ids[i], rideogram_ids[i])

        n2_chrom = (N2_all_df.at[n2_id, 'SeqChr'])
        cb_chrom = (CB_all_df.at[cb_id, 'SeqChr'])
    
        n2_start = int(N2_all_df.at[n2_id, 'Start'])
        n2_stop = int(N2_all_df.at[n2_id, 'Stop'])

        cb_start = int(CB_all_df.at[cb_id, 'Start'])
        cb_stop = int(CB_all_df.at[cb_id, 'Stop'])

        # define color for each TE line
        if sort_by_class:
            if te_class == 'DNATransposon':
              color = '204517' # color DNA TEs blue
            elif te_class == 'Retrotransposon':
              color = 'FF0000' # color retro TEs red
        else:
            color = '000000' # if sort_by_color False then all lines black

        # include TE ids in outfile?
        if include_ids == False:
            line = [ n2_chrom, n2_start, n2_stop, cb_chrom, cb_start, cb_stop, color]
        else: 
            line = [n2_id, n2_chrom, n2_start, n2_stop, cb_id, cb_chrom, cb_start, cb_stop]
            
        matching_tsv_list.append(line)

    if include_ids == False: 
        matching_tsv_df = pd.DataFrame(matching_tsv_list, columns = [ 'n2_chrom', 'n2_start', 'n2_stop',  'cb_chrom', 'cb_start', 'cb_stop', 'color'])    
    else: 
        matching_tsv_df = pd.DataFrame(matching_tsv_list, columns = ['n2_id', 'n2_chrom', 'n2_start', 'n2_stop', 'cb_id', 'cb_chrom', 'cb_start', 'cb_stop'])    


    if to_save and include_ids:
        matching_tsv_df.to_csv("matchingTEs_info_ids.tsv", sep="\t", index=False)
    elif to_save and not include_ids:
        matching_tsv_df.to_csv("matchingTEs_info_noids.tsv", sep="\t", index=False)

    return matching_tsv_df

In [51]:
matching_tsv_df = create_tsv_file(matching_df, False, True, True)

## 8.2. <a id='toc8_2_'></a>[FX: `create_tsv_file_specfams`](#toc0_)
- same basic function as `create_tsv_file` but instead creates separate tsv files for every specific TE family listed in the `spec_families` list within the function (this can be changed to have whichever specific families you want)

In [67]:
def create_tsv_file_specfams(matching_df, to_save: bool = False): 
    """ 
    matching_df: dataframe created above
    to_save: bool, whether or not to save the graph as a pdf (if False, graph will only be output into the notebook)
    """
    spec_families = ['CMC', 'Zator', 'hAT', 'Gypsy', 'Copia', 'Sola', 'Tc1-Mariner',
        'MITE', 'Helitron', 'Novosib', 'LINE', 'ERV', 'SINE']

    df_ids = ['seq1', 'seq2', 'seq3', 'seq4', 'seq5', 'seq6']
    rideogram_ids = [1, 2, 3, 4, 5, 6]

    for i in range(len(spec_families)):
        
        family = spec_families[i]
        
        matching_tsv_list = []
        
        for index, row in matching_df.iterrows():
            
            n2_id = int(row['N2_id'])
            cb_id = (row['CB_id'])

            if N2_all_df.at[n2_id, 'Family'] == family:

                # replace chromosome names with numbers for RIdeogram      
                for i in range(6):
                    
                    N2_all_df['SeqChr'] = N2_all_df['SeqChr'].replace(df_ids[i], rideogram_ids[i])
                    CB_all_df['SeqChr'] = CB_all_df['SeqChr'].replace(df_ids[i], rideogram_ids[i])

                n2_chrom = (N2_all_df.at[n2_id, 'SeqChr'])
                cb_chrom = (CB_all_df.at[cb_id, 'SeqChr'])

                n2_start = int(N2_all_df.at[n2_id, 'Start'])
                n2_stop = int(N2_all_df.at[n2_id, 'Stop'])

                cb_start = int(CB_all_df.at[cb_id, 'Start'])
                cb_stop = int(CB_all_df.at[cb_id, 'Stop'])

                color = '000000'

                line = [n2_chrom, n2_start, n2_stop, cb_chrom, cb_start, cb_stop, color]

                matching_tsv_list.append(line)

        
        matching_tsv_df = pd.DataFrame(matching_tsv_list)  

        if to_save:  
            matching_tsv_df.to_csv(f"matchingTEs_{family}.tsv", sep="\t", index=False)

        return matching_tsv_df
